# Clustering analysis and PCA #

## Missing R packages##

When running the scripts in this section, if you get an error saying the gplots package has not been installed, you have two options: (1) either run the following command to add a pre-downloaded version of the package to the **$R_LIBS** environment variable...

In [1]:
!export R_LIBS=$R_LIBS:/users/avanti/R/x86_64-unknown-linux-gnu-library/3.0/

...or install the package locally by  running install.packages("gplots"): 

In [2]:
%%bash 
R --no-save 
install.packages("gplots",repos='http://cran.us.r-project.org')
q()


R version 3.0.2 (2013-09-25) -- "Frisbee Sailing"
Copyright (C) 2013 The R Foundation for Statistical Computing
Platform: x86_64-pc-linux-gnu (64-bit)

R is free software and comes with ABSOLUTELY NO WARRANTY.
You are welcome to redistribute it under certain conditions.
Type 'license()' or 'licence()' for distribution details.

R is a collaborative project with many contributors.
Type 'contributors()' for more information and
'citation()' on how to cite R or R packages in publications.

Type 'demo()' for some demos, 'help()' for on-line help, or
'help.start()' for an HTML browser interface to help.
Type 'q()' to quit R.

> install.packages("gplots",repos='http://cran.us.r-project.org')


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)
Warning in install.packages("gplots", repos = "http://cran.us.r-project.org") :
  'lib = "/usr/local/lib/R/site-library"' is not writable
Error in install.packages("gplots", repos = "http://cran.us.r-project.org") : 
  unable to install packages
Execution halted


## The process_peaks.sh script ##

Cluster analysis is a simple way to visualize patterns in the data. By clustering peaks according to their signal across different time points, we may find groups of peaks that have similar behavior across these time points. By clustering samples according to their signal across peaks, we can perform a simple sanity check of data quality ‐ samples of the same time point should cluster together.

We have developed a script to perform all the following steps:

In [ ]:
! ${SRC_DIR}/process_peaks.sh

## Heatmap generation ##

Note that the last step in the script above is to generate heatmaps. This step may fail if X11 is not installed. One solution is to copy the foldChange.tab file to your local computer, and then run:

**[path/to/]visualize_clusters.R foldChange.tab foldChange.png**

Note that visualize_clusters.R is obtained from the github repository of scripts: https://github.com/kundajelab/training_camp/blob/master/src/visualize_clusters.R

## Separating out wt and ko ##

The strong difference between wt and ko may make the heatmap hard to discern. One solution is to separate out the wt and the ko samples, and call visualise_clusters.R on each one. You can do this with:

In [ ]:
! perl -lane 'if ($. == 1) {@titleIdxs = grep {$F[$_] =~ /wt/} 0..$#F; @idxs = map {$_+1} @titleIdxs; print "@F[@titleIdxs]"} else {print "@F[0,@idxs]"}' foldChange.tab > wt_foldChange.tab
! perl -lane 'if ($. == 1) {@titleIdxs = grep {$F[$_] =~ /ko/} 0..$#F; @idxs = map {$_+1} @titleIdxs; print "@F[@titleIdxs]"} else {print "@F[0,@idxs]"}' foldChange.tab > ko_foldChange.tab

Then use the steps you used to visualise the heatmaps, but supply **wt_foldChange.tab** or **ko_foldChange.tab** where you previously supplied **foldChange.tab**.

## PCA ##

PCA (Principal Component Analysis) is a way to identify the primary directions of variation in the data. It can also be used for very coarse-grained clustering of samples; similar samples will have similar coordinates along the principal axes.

We will perform PCA on foldChange.tab. The first step is to clean up the column labels in foldChange.tab with the following perl one-liner:

In [ ]:
%cd $WORK_DIR/results/signal
!perl -i".bak" -pe '$_ = $.==1 ? do {$_ =~ s/\/[^\s]+\///g; $_ =~ s/\"//g; $_ =~ s/\-/\./g; $_ =~ s/PooledReps_Sample/samp/g; $_} : $_' foldChange.tab

We will now do PCA. We treat each sample as a single point in a very high dimensional space (where the dimensionality is equal to the number of genes the vary), and then we will perform dimensionality reduction in this space (if you get an X11 error, log into R studio and follow the instructions at the end):

In [ ]:
%cd $WORK_DIR/results/signal
!$SRC_DIR/doPCA.R foldChange.tab

This script will produce PCA_sdev.png, which shows the standard deviation explained by each of the principle components. Since there are only 12 datapoints, the effective dimensionality of our data is 12, even though there are thousands of genes; this is why there are only 12 PCs.

It also produces PC_[x]_vs_[y].png for components 1..3. How do you interpret the different principle components?

Finally, it produces the files pc[x]_rotation.txt for components 1..3, which show the contribution of each peak to the direction of the principle component; this file can be used to get a sense of which peaks are critical in defining the principle components, and in which direction (positive or negative). One interesting analysis we can do with these files is to sort the peaks by their contribution to the principle component in ascending or descending order, map the peaks to their nearest genes, and then used the ranked list with software such as GOrilla which accept a ranked list of genes and output which GO terms are overrepresented towards the top: (http://cbl-gorilla.cs.technion.ac.il/)

The following commands will sort the genes by their contribution to each principle component and then map them to their nearest gene:



In [ ]:
%cd $WORK_DIR/results/signal

In [ ]:
%%bash
for pcFile in `ls pc*_rotation.txt`; do
    theBase=`basename ${pcFile}`
    cat $pcFile | sort -k 2r > "ascending_"$theBase
    cat $pcFile | sort -k 2rg > "descending_"$theBase
done
$SRC_DIR/mapToNearestPeak.py --sigPeakInputFiles ascending*.txt descending*.txt --peaks2genesFile /tc2015/avanti/results/peaks/peaks2genes.bed

You can then copy the mapped files to your local computer with:

In [ ]:
!scp [yourUsername]@wotan.stanford.edu:/tc2015/[yourUsername]/results/signal/nearestGenes*pc* .